In [2]:
import os
import pandas as pd

In [4]:
os.chdir("/Users/david/Documents/Informatics/TCGA_Melanoma_RNASeq")

In [19]:
#epi_mes_score = pd.read_csv("outputs/epi_mes_signature_scores.csv", usecols=lambda x: x != "Unnamed: 0")
epi_mes_score = pd.read_csv("outputs/epi_mes_signature_scores.csv", usecols = ["Sample_ID", "ssGSEA_patient_epi_score", "ssGSEA_patient_mes_score"])
epi_mes_score.head()

,Sample_ID,ssGSEA_patient_epi_score,ssGSEA_patient_mes_score
0,mel001,0.149018,0.601737
1,mel002,0.201855,0.631269
2,mel003,0.157802,0.753784
3,mel004,0.191706,0.554946
4,mel005,0.067251,0.777031


In [9]:
wgcna_module_values = pd.read_csv("outputs/WGCNA_module_eigenvalues.csv")
wgcna_module_values.rename({"Unnamed: 0" : "Sample_ID"}, axis = "columns", inplace = True)
wgcna_module_values.head()

,Sample_ID,MEblue,MEturquoise,MEyellow,MEgreen,MEmagenta,MEbrown,MEsalmon,MElightcyan,MEblack,MEgreenyellow,MEred,MEmidnightblue,MEtan,MEcyan,MEpink,MEpurple,MEgrey
0,mel354,0.020971,0.005956,-0.086223,-0.048382,-0.057826,0.023024,-0.001378,-0.012002,0.023168,-0.033896,-0.035925,0.016831,0.020362,0.051121,-0.036941,0.035503,-0.045974
1,mel432,0.044355,0.020178,0.057323,-0.032854,-0.004383,-0.024271,0.002157,-0.018926,-0.028900,-0.014615,-0.062450,-0.052693,-0.031941,-0.040906,-0.017927,-0.009794,-0.009022
2,mel243,0.028841,-0.008393,0.045201,0.043760,0.036315,-0.022660,0.002516,0.032038,0.055258,0.097136,0.003865,0.044203,0.013156,-0.049340,-0.075039,-0.015590,0.036642
3,mel321,-0.084911,-0.038733,-0.078051,0.028075,-0.022807,0.056430,0.015542,-0.004794,0.020139,-0.006115,-0.000501,-0.003662,0.027545,0.019350,0.050113,0.057358,-0.048247
4,mel218,-0.007241,0.042464,-0.071064,-0.040436,-0.052362,0.012555,-0.015324,-0.017386,-0.008267,-0.037702,0.044787,0.018398,0.018849,0.031989,0.059126,0.018955,-0.093132


In [11]:
deconvolutions = pd.read_csv("outputs/epic_deconvolutions.csv", skiprows = 1)
#deconvolutions.rename({"cell_type" : "Sample_ID"}, axis = "columns", inplace = True)
deconvolutions.columns = ["Sample_ID", "Bcells", "CAFs", "CD4_Tcells", "CD8_Tcells", "Endothelial", "Macrophages", "NK_cells", "Other_cells"]
deconvolutions.head()

,Sample_ID,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NK_cells,Other_cells
0,mel354,1.025579e-10,0.023539,1.045116e-07,7.382235e-08,0.109605,0.005625,3.814432e-05,0.861194
1,mel432,5.479552e-02,0.022260,2.640453e-01,1.779707e-01,0.113383,0.015683,4.243849e-11,0.351862
2,mel243,6.988778e-02,0.019769,2.511736e-01,9.669438e-02,0.149864,0.011853,5.487604e-10,0.400758
3,mel321,5.082280e-09,0.026713,5.343004e-02,3.707480e-08,0.178638,0.007351,6.497756e-07,0.733868
4,mel218,1.586768e-05,0.023621,1.519758e-01,9.125375e-05,0.122105,0.010320,4.908923e-09,0.691871


In [22]:
#could use reduce function here
sample_data = pd.merge(epi_mes_score, wgcna_module_values, on = "Sample_ID")
sample_data = pd.merge(sample_data, deconvolutions, on = "Sample_ID")
sample_data.head()


,Sample_ID,ssGSEA_patient_epi_score,ssGSEA_patient_mes_score,MEblue,MEturquoise,MEyellow,MEgreen,MEmagenta,MEbrown,MEsalmon,...,MEpurple,MEgrey,Bcells,CAFs,CD4_Tcells,CD8_Tcells,Endothelial,Macrophages,NK_cells,Other_cells
0,mel001,0.149018,0.601737,-0.027554,0.008308,0.065262,-0.011695,0.006539,-0.007475,0.023422,...,-0.022201,-0.006652,1.006166e-01,0.022282,0.318466,0.148652,0.165866,0.016016,1.873212e-09,0.228101
1,mel002,0.201855,0.631269,-0.034268,0.011327,0.025679,0.009168,-0.003292,0.001200,0.002366,...,0.039837,-0.038442,4.633967e-02,0.026364,0.223059,0.121218,0.156134,0.014540,2.535559e-10,0.412345
2,mel003,0.157802,0.753784,0.036205,0.020433,0.107598,0.059451,0.042216,-0.096866,0.005888,...,-0.044652,0.002794,8.955805e-02,0.026561,0.388977,0.195644,0.198778,0.015364,1.378513e-10,0.085119
3,mel004,0.191706,0.554946,-0.094093,-0.053224,-0.016999,-0.008038,-0.064231,0.086740,0.084928,...,0.079763,-0.083695,8.181303e-09,0.030694,0.264179,0.024643,0.135870,0.014467,1.959118e-09,0.530147
4,mel005,0.067251,0.777031,0.040886,0.017279,0.038992,0.133210,0.125685,-0.061881,-0.007971,...,-0.115584,0.054426,8.053216e-03,0.029098,0.275838,0.084256,0.205182,0.012719,1.654998e-10,0.384855
